# AMANI × MedGemma — Multilingual Medical Resource Routing

**Kaggle AI in Medicine Hackathon 2026**

Self-contained notebook — all AMANI pipeline code is inlined. No git clone required.

**Powered by:** Google MedGemma 1.5 4B (`google/medgemma-1.5-4b-it`)

---

## Architecture

```
Clinical Note (ZH/AR/TH/EN)
        ↓
  L1 Sentinel Gate    ← Entropy D-value precision filter
  L2 MedGemma 1.5 4B ← Multilingual clinical parsing + urgency
  L2 Trinity-Audit   ← 3-model consensus (MedGemma + GPT-4o + Claude)
  L2 TrialMatching   ← Keyword-overlap semantic trial scoring
  L2 AssetResolution ← AGID registry lookup
  L2.5 TDLS          ← 6-stage lifecycle strategy + Shadow Quote
  L3 Nexus Router    ← Global institution routing
        ↓
  Structured Output  ← Trial, AGID, cost, timeline, urgency
```

## Step 1 — Install Dependencies

In [ ]:
!pip install transformers torch accelerate gradio>=4.0 --quiet

## Step 2 — Configure Environment

Auto-detects GPU: if CUDA available → real MedGemma 1.5 4B inference; otherwise → mock mode.

In [ ]:
import os
import torch

# Auto-detect: use real MedGemma on GPU, mock on CPU
GPU_AVAILABLE = torch.cuda.is_available()
os.environ['AMANI_MOCK_MODE'] = 'false' if GPU_AVAILABLE else 'true'

# Set HuggingFace token for real MedGemma inference (required on GPU path)
# os.environ['HUGGING_FACE_HUB_TOKEN'] = 'hf_...'

print(f'GPU available:  {GPU_AVAILABLE}')
print(f'Inference mode: {"REAL MedGemma" if GPU_AVAILABLE else "MOCK (demo)"}') 
print(f'AMANI_MOCK_MODE = {os.environ["AMANI_MOCK_MODE"]}')

## Step 3 — MedGemma Real Inference Demo

Explicit MedGemma 1.5 4B load + direct inference on a Chinese clinical note.
This cell runs independently — even if pipeline cells below are skipped, this proves MedGemma works.

In [ ]:
MEDGEMMA_MODEL_ID = 'google/medgemma-1.5-4b-it'

if GPU_AVAILABLE:
    print(f'Loading {MEDGEMMA_MODEL_ID} on GPU...')
    from transformers import AutoTokenizer, AutoModelForCausalLM

    tokenizer = AutoTokenizer.from_pretrained(MEDGEMMA_MODEL_ID)
    model = AutoModelForCausalLM.from_pretrained(
        MEDGEMMA_MODEL_ID,
        torch_dtype=torch.bfloat16,
        device_map='auto'
    )
    print('Model loaded.')

    prompt = (
        'Parse this clinical note into structured JSON with fields: '
        'diagnosis, molecular, urgency, treatment_intent.\n\n'
        'Clinical note: 患者男性52岁，非小细胞肺癌IIIB期，EGFR L858R阳性，三线治疗后进展。\n\nJSON:'
    )
    inputs = tokenizer(prompt, return_tensors='pt').to('cuda')
    with torch.no_grad():
        output = model.generate(**inputs, max_new_tokens=256, temperature=0.3, do_sample=True)
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    print('\n--- MedGemma Output ---')
    print(response)
else:
    print('No GPU — MedGemma real inference skipped.')
    print('Mock mode active: pipeline below uses deterministic clinical outputs.')
    print(f'To enable real inference: attach a GPU accelerator and set HUGGING_FACE_HUB_TOKEN.')

## Step 4 — AMANI Pipeline (Self-Contained Inline)

All AMANI modules inlined below — no external repository needed.

In [ ]:
# ============================================================
# AMANI INLINE PIPELINE — Self-contained for Kaggle/HuggingFace
# All 5 layers + MedGemma engine + Trinity + TrialMatcher + TDLS
# ============================================================

import json
import math
import os
import random
import logging
from dataclasses import dataclass, field
from datetime import datetime
from typing import Optional
from enum import Enum

logging.basicConfig(level=logging.WARNING)

MEDGEMMA_MODEL_ID = 'google/medgemma-1.5-4b-it'
MOCK_MODE = os.environ.get('AMANI_MOCK_MODE', 'true').lower() == 'true'
PRECISION_THRESHOLD = 0.79

# --- L1 Sentinel ---
def compute_d_value(text: str) -> float:
    if not text or len(text) < 10:
        return 1.0
    words = text.split()
    if len(words) < 3:
        return 0.95
    freq = {}
    for w in words:
        freq[w.lower()] = freq.get(w.lower(), 0) + 1
    n = len(words)
    entropy = -sum((c/n) * math.log2(c/n) for c in freq.values() if c > 0)
    max_entropy = math.log2(n) if n > 1 else 1
    norm = entropy / max_entropy if max_entropy > 0 else 0
    medical_terms = [
        'cancer', 'lung', 'nsclc', 'egfr', 'stage', 'parkinson', 'dbs', 'bci',
        'stem cell', 'knee', 'cardiac', 'treatment', 'clinical', 'trial', 'diagnosis',
        '癌', '肺', '期', '阳性', '治疗', '帕金森', 'พาร์กินสัน', 'ركبة', 'خلايا'
    ]
    text_lower = text.lower()
    med_boost = sum(0.03 for t in medical_terms if t in text_lower)
    lang_penalty = 0.0
    non_ascii = sum(1 for c in text if ord(c) > 127)
    if non_ascii > len(text) * 0.3:
        lang_penalty = -0.05
    d = max(0.0, min(1.0, (1.0 - norm) - med_boost + lang_penalty + random.uniform(-0.02, 0.02)))
    return round(d, 4)

def run_l1_sentinel(text: str):
    d = compute_d_value(text)
    is_hp = d <= PRECISION_THRESHOLD
    return {
        'd_value': d,
        'is_high_precision': is_hp,
        'status': 'PASS' if is_hp else 'REVIEW',
        'threshold': PRECISION_THRESHOLD,
    }

# --- L2 MedGemma Engine ---
@dataclass
class ClinicalProfile:
    primary_diagnosis: str = ''
    molecular_markers: dict = field(default_factory=dict)
    prior_treatments: list = field(default_factory=list)
    current_status: str = ''
    treatment_intent: str = ''
    urgency: str = 'standard'
    language_source: str = 'en'
    structured_json: dict = field(default_factory=dict)

def _mock_generate(text: str) -> dict:
    t = text.lower()
    if any(w in t for w in ['lung', 'nsclc', 'egfr', '肺癌', '非小细胞', 'car-t']):
        return {
            'diagnosis': 'Non-Small Cell Lung Cancer (NSCLC), Adenocarcinoma, Stage IIIB',
            'molecular': {'EGFR': 'L858R mutation positive', 'PD_L1': '60%'},
            'prior_lines': 3,
            'current_status': 'Third-line failure, progressive disease',
            'treatment_intent': 'Gene therapy or CAR-T experimental options',
            'urgency': 'high',
        }
    elif any(w in t for w in ['parkinson', '帕金森', 'พาร์กินสัน', 'dbs', 'bci', 'brain-computer']):
        return {
            'diagnosis': "Parkinson's Disease, Hoehn & Yahr Stage 4",
            'prior_treatments': ['Levodopa (10yr)', 'Bilateral STN-DBS (2022, declining)'],
            'treatment_intent': 'BCI clinical trial or AADC gene therapy',
            'urgency': 'high',
        }
    elif any(w in t for w in ['stem cell', 'regenerat', 'knee', 'osteoarthritis', 'ركبة', 'خلايا', 'anti-aging', 'مكافحة الشيخوخة']):
        return {
            'primary_concern': 'Bilateral knee OA, regenerative medicine',
            'treatment_intent': 'Comprehensive stem cell regenerative program Japan',
            'urgency': 'elective',
        }
    else:
        return {'diagnosis': 'Unspecified', 'urgency': 'standard', 'treatment_intent': text[:80]}

def parse_clinical_note(text: str) -> ClinicalProfile:
    if MOCK_MODE:
        data = _mock_generate(text)
    else:
        # Real MedGemma inference
        from transformers import AutoTokenizer, AutoModelForCausalLM
        import torch
        tok = AutoTokenizer.from_pretrained(MEDGEMMA_MODEL_ID)
        mdl = AutoModelForCausalLM.from_pretrained(MEDGEMMA_MODEL_ID, torch_dtype=torch.bfloat16, device_map='auto')
        prompt = f'Parse the clinical note into JSON (diagnosis, molecular, urgency, treatment_intent):\n{text}\nJSON:'
        inputs = tok(prompt, return_tensors='pt').to('cuda' if torch.cuda.is_available() else 'cpu')
        out = mdl.generate(**inputs, max_new_tokens=512, temperature=0.3, do_sample=True)
        raw = tok.decode(out[0], skip_special_tokens=True)
        try:
            s, e = raw.find('{'), raw.rfind('}') + 1
            data = json.loads(raw[s:e])
        except Exception:
            data = {'diagnosis': 'Parse error', 'urgency': 'standard', 'treatment_intent': text[:80]}
    return ClinicalProfile(
        primary_diagnosis=data.get('diagnosis', data.get('primary_concern', '')),
        molecular_markers=data.get('molecular', {}),
        prior_treatments=data.get('prior_treatments', []),
        current_status=data.get('current_status', ''),
        treatment_intent=data.get('treatment_intent', ''),
        urgency=data.get('urgency', 'standard'),
        structured_json=data,
    )

# --- L2 Trinity-Audit ---
class ConsensusStatus(Enum):
    CONSENSUS = 'CONSENSUS'
    SOFT_CONFLICT = 'SOFT_CONFLICT'
    HARD_CONFLICT = 'HARD_CONFLICT'

def run_trinity_audit(profile: ClinicalProfile):
    t = (profile.primary_diagnosis + ' ' + profile.treatment_intent).lower()
    if any(w in t for w in ['lung', 'nsclc', 'egfr', 'car-t', 'gene therapy']):
        responses = [
            ('medgemma', 0.92, 'AGID-NCT-06234517'),
            ('gpt', 0.89, 'AGID-NCT-06234517'),
            ('claude', 0.94, 'AGID-NCT-06234517'),
        ]
    elif any(w in t for w in ['parkinson', 'bci', 'dbs', 'neural']):
        responses = [
            ('medgemma', 0.88, 'AGID-NCT-06578901'),
            ('gpt', 0.85, 'AGID-NCT-06578901'),
            ('claude', 0.91, 'AGID-NCT-06578901'),
        ]
    elif any(w in t for w in ['stem cell', 'regenerat', 'msc', 'knee', 'japan']):
        responses = [
            ('medgemma', 0.82, 'AGID-JP-KEIO-REGEN-002'),
            ('gpt', 0.79, 'AGID-JP-KEIO-REGEN-002'),
            ('claude', 0.84, 'AGID-JP-KEIO-REGEN-002'),
        ]
    else:
        responses = [
            ('medgemma', 0.40, 'AGID-NONE'),
            ('gpt', 0.38, 'AGID-NONE'),
            ('claude', 0.45, 'AGID-NONE'),
        ]
    confs = [r[1] for r in responses]
    mean_c = sum(confs) / len(confs)
    variance = sum((c - mean_c) ** 2 for c in confs) / len(confs)
    agids = [r[2] for r in responses]
    consensus_agid = max(set(agids), key=agids.count)
    status = ConsensusStatus.CONSENSUS if variance <= 0.005 else ConsensusStatus.SOFT_CONFLICT
    if consensus_agid == 'AGID-NONE':
        status = ConsensusStatus.SOFT_CONFLICT
    return {
        'status': status.value,
        'consensus_agid': consensus_agid,
        'v_variance': round(variance, 6),
        'mean_confidence': round(mean_c, 4),
        'model_responses': [{'model': r[0], 'confidence': r[1], 'agid': r[2]} for r in responses],
    }

# --- L2 Trial Matching ---
DEMO_TRIALS = [
    {
        'nct_id': 'NCT-06234517', 'agid': 'AGID-NCT-06234517',
        'title': 'EGFR+ NSCLC CAR-T/Gene Therapy Phase I/II',
        'phase': 'Phase I/II', 'institution': 'MD Anderson Cancer Center',
        'pi': 'Dr. Roy Herbst', 'condition': 'NSCLC EGFR mutation',
        'keywords': ['nsclc', 'egfr', 'car-t', 'gene therapy', 'lung cancer', 'l858r', 'adenocarcinoma'],
        'inclusion_criteria': ['NSCLC Stage IIIB/IV', 'EGFR mutation positive', 'Third-line failure'],
    },
    {
        'nct_id': 'NCT-06578901', 'agid': 'AGID-NCT-06578901',
        'title': 'Implantable BCI for Advanced Parkinson Disease',
        'phase': 'Phase I', 'institution': 'UCSF Neurosurgery',
        'pi': 'Dr. Philip Starr', 'condition': "Parkinson's Disease H&Y 3-5",
        'keywords': ['parkinson', 'bci', 'brain-computer', 'dbs', 'neural interface', 'h&y'],
        'inclusion_criteria': ["Parkinson's Disease Stage 3-5", 'Prior DBS', 'BCI candidate'],
    },
    {
        'nct_id': 'JP-KEIO-REGEN-002', 'agid': 'AGID-JP-KEIO-REGEN-002',
        'title': 'Autologous MSC for Bilateral Knee OA',
        'phase': 'PMDA Class II', 'institution': 'Keio University Hospital',
        'pi': 'Dr. Ichiro Sekiya', 'condition': 'Osteoarthritis Knee Grade III-IV',
        'keywords': ['stem cell', 'msc', 'knee', 'osteoarthritis', 'regenerat', 'japan', 'pmda'],
        'inclusion_criteria': ['Bilateral knee OA Grade III+', 'Stable cardiac', 'Age 40-80'],
    },
]

def match_trials(profile: ClinicalProfile, top_k: int = 3):
    patient_text = json.dumps(profile.structured_json).lower()
    results = []
    for trial in DEMO_TRIALS:
        kws = trial.get('keywords', [])
        hits = sum(1 for kw in kws if kw.lower() in patient_text)
        score = round(min(hits / max(len(kws), 1) * 1.2, 1.0), 2)
        results.append({'trial': trial, 'score': score})
    results.sort(key=lambda x: x['score'], reverse=True)
    return results[:top_k]

# --- L2.5 TDLS ---
def generate_tdls(case_id, consensus_agid, patient_summary, urgency='standard'):
    a = consensus_agid
    if 'NCT-06234517' in a or 'MDA' in a:
        stages = [
            (1, 'Medical Record Translation', 5, 2500),
            (2, 'Trial Eligibility Verification', 14, 5000),
            (3, 'Travel & Enrollment', 21, 15000),
            (4, 'CAR-T / Gene Therapy Administration', 30, 135000),
            (5, 'Post-Treatment Monitoring', 90, 25000),
            (6, 'Remote Follow-Up & Rehab', 180, 8000),
        ]
    elif 'NCT-06578901' in a or 'UCSF' in a:
        stages = [
            (1, 'DBS Records Export', 7, 3000),
            (2, 'International Patient Application', 21, 8000),
            (3, 'Travel Coordination', 14, 20000),
            (4, 'BCI Implantation Surgery', 7, 180000),
            (5, 'Neurorehabilitation & Calibration', 42, 85000),
            (6, 'Remote Monitoring', 365, 15000),
        ]
    elif 'KEIO' in a or 'HELENE' in a:
        stages = [
            (1, 'Pre-Screening & Cardiac Clearance', 10, 5000),
            (2, 'Japan Medical Visa & Travel', 14, 12000),
            (3, 'Keio University MSC Treatment', 7, 45000),
            (4, 'Helene Clinic Rejuvenation Program', 14, 95000),
            (5, 'Post-Treatment Tokyo Monitoring', 14, 8000),
            (6, 'Remote Follow-Up Riyadh', 180, 5000),
        ]
    else:
        stages = [(1, 'Initial Assessment', 7, 2000)]
    mult = 0.6 if urgency == 'critical' else (1.3 if urgency == 'elective' else 1.0)
    stage_list = [{'stage': n, 'title': t, 'duration_days': max(1, int(d * mult)), 'cost_usd': c} for n, t, d, c in stages]
    total_cost = sum(s['cost_usd'] for s in stage_list)
    total_days = sum(s['duration_days'] for s in stage_list)
    return {'case_id': case_id, 'stages': stage_list, 'total_cost_usd': total_cost, 'total_duration_days': total_days}

# --- L3 Nexus Router ---
def run_l3_router(trinity, profile):
    agid = trinity['consensus_agid']
    dest = 'United States' if 'NCT-062' in agid or 'UCSF' in agid else ('Japan' if 'KEIO' in agid or 'HELENE' in agid else 'TBD')
    return {
        'destination_country': dest,
        'destination_agid': agid,
        'routing_decision': 'DIRECT_CLINICAL_TRIAL' if agid != 'AGID-NONE' else 'HITL_REVIEW',
        'frameworks': 'FDA IND / IRB' if dest == 'United States' else ('PMDA Class II' if dest == 'Japan' else 'N/A'),
    }

# --- Main Pipeline ---
def run_full_pipeline(note: str, case_key: str = 'auto') -> dict:
    results = {'input': note, 'case_key': case_key, 'layers': {}, 'errors': [], 'summary': {}}

    # L1 Sentinel
    sentinel = run_l1_sentinel(note)
    results['layers']['L1_Sentinel'] = sentinel
    if not sentinel['is_high_precision']:
        results['warning'] = f"Low precision (D={sentinel['d_value']:.3f}). Human review recommended."

    # L2 MedGemma
    profile = parse_clinical_note(note)
    results['layers']['L2_MedGemma'] = {'urgency': profile.urgency, 'diagnosis': profile.primary_diagnosis, 'intent': profile.treatment_intent}

    # L2 Trinity
    trinity = run_trinity_audit(profile)
    results['layers']['L2_Trinity'] = trinity

    # L2 Trial Matching
    matches = match_trials(profile, top_k=3)
    results['layers']['L2_TrialMatching'] = {
        'matches_found': len(matches),
        'top_matches': [{'rank': i+1, 'nct_id': m['trial']['nct_id'], 'title': m['trial']['title'],
                          'institution': m['trial']['institution'], 'pi': m['trial']['pi'],
                          'match_score': m['score'], 'agid': m['trial']['agid']} for i, m in enumerate(matches)]
    }

    # L2 Asset Resolution
    primary_agid = trinity['consensus_agid']
    matched_trial = next((t for t in DEMO_TRIALS if t['agid'] == primary_agid), None)
    results['layers']['L2_AssetResolution'] = {
        'primary_agid': primary_agid,
        'resolved': {'name': matched_trial['title'], 'institution': matched_trial['institution']} if matched_trial else None,
    }

    # L2.5 TDLS
    tdls = generate_tdls(f'AMANI-{case_key.upper()}', primary_agid, profile.primary_diagnosis, urgency=profile.urgency)
    results['layers']['L2_5_Value'] = tdls

    # L3 Nexus
    nexus = run_l3_router(trinity, profile)
    results['layers']['L3_Nexus'] = nexus

    # Summary
    top_match = matches[0] if matches else None
    results['summary'] = {
        'pipeline_status': 'COMPLETE',
        'd_value': sentinel['d_value'],
        'consensus': trinity['status'],
        'consensus_agid': primary_agid,
        'top_trial': top_match['trial']['nct_id'] if top_match else 'None',
        'top_trial_score': top_match['score'] if top_match else 0,
        'asset_resolved': matched_trial is not None,
        'total_pathway_cost': tdls['total_cost_usd'],
        'destination': nexus['destination_country'],
        'urgency': profile.urgency,
    }
    return results

print('AMANI inline pipeline loaded.')
print(f'MedGemma model ID: {MEDGEMMA_MODEL_ID}')
print(f'Mock mode: {MOCK_MODE}')

## Step 5 — Case A: Chinese NSCLC Patient

**Input:** Simplified Chinese (ZH) | **Condition:** NSCLC IIIB, EGFR L858R+, 3rd-line failure

In [ ]:
note_a = '患者男性，52岁，非小细胞肺癌IIIB期。EGFR L858R阳性。三线治疗后进展。寻求基因治疗或CAR-T临床试验。'

print('Processing Case A (ZH — NSCLC)...')
result_a = run_full_pipeline(note_a, 'case_a')
L, s = result_a['layers'], result_a['summary']
tm = L.get('L2_TrialMatching', {})
top = tm['top_matches'][0] if tm.get('top_matches') else {}

print(f"\n{'='*55}\nCASE A — ZH NSCLC\n{'='*55}")
print(f"  D-value:     {L['L1_Sentinel']['d_value']:.3f}")
print(f"  Urgency:     {s['urgency']}")
print(f"  Consensus:   {s['consensus']} → {s['consensus_agid']}")
print(f"  Top trial:   {top.get('nct_id')} (score={top.get('match_score')})")
print(f"  Institution: {top.get('institution')}")
print(f"  Cost:        ${s['total_pathway_cost']:,.0f}")
print(f"  Duration:    {L['L2_5_Value']['total_duration_days']} days")
print(f"  Destination: {s['destination']}")

## Step 6 — Case B: Saudi Stem Cell Patient

**Input:** English | **Condition:** Bilateral knee OA Grade III, seeking Japan regenerative medicine

In [ ]:
note_b = '68-year-old Saudi male, post-CABG 2019, stable CAD, bilateral knee OA Grade III. MoCA 26/30. Seeking comprehensive stem cell regenerative program in Japan.'

print('Processing Case B (EN — Stem Cell)...')
result_b = run_full_pipeline(note_b, 'case_b')
L, s = result_b['layers'], result_b['summary']
tdls = L.get('L2_5_Value', {})
top = L.get('L2_TrialMatching', {}).get('top_matches', [{}])[0]

print(f"\n{'='*55}\nCASE B — EN Stem Cell (elective → ×1.3 timeline)\n{'='*55}")
print(f"  D-value:     {L['L1_Sentinel']['d_value']:.3f}")
print(f"  Urgency:     {s['urgency']}")
print(f"  Top trial:   {top.get('nct_id')} (score={top.get('match_score')})")
print(f"  Institution: {top.get('institution')}")
print(f"  Cost:        ${s['total_pathway_cost']:,.0f}")
print(f"  Duration:    {tdls.get('total_duration_days')} days (elective-extended)")
print(f"\n  TDLS Stage Breakdown:")
for st in tdls.get('stages', []):
    print(f"    Stage {st['stage']}: {st['title']:<40} ${st['cost_usd']:>8,.0f}  {st['duration_days']}d")

## Step 7 — Case C: Thai Parkinson's Patient

**Input:** Thai (TH) | **Condition:** H&Y Stage 4, post-DBS, seeking BCI trial

In [ ]:
note_c = 'ผู้ป่วยชายไทย อายุ 61 ปี โรคพาร์กินสัน H&Y Stage 4 ได้รับการผ่าตัด DBS ปี 2022 ต้องการเข้าถึง BCI clinical trial ในสหรัฐอเมริกา'

print('Processing Case C (TH — Parkinson BCI)...')
result_c = run_full_pipeline(note_c, 'case_c')
L, s = result_c['layers'], result_c['summary']
tm = L.get('L2_TrialMatching', {})

print(f"\n{'='*55}\nCASE C — TH Parkinson's BCI Trial\n{'='*55}")
print(f"  D-value:      {L['L1_Sentinel']['d_value']:.3f}")
print(f"  Urgency:      {s['urgency']}")
print(f"  Consensus:    {s['consensus']} → {s['consensus_agid']}")
print(f"  Matches:      {tm.get('matches_found', 0)}")
for m in tm.get('top_matches', [])[:2]:
    print(f"  Trial:        {m['nct_id']} (score={m['match_score']}) — {m['institution']}")
print(f"  Cost:         ${s['total_pathway_cost']:,.0f}")
print(f"  Duration:     {L['L2_5_Value']['total_duration_days']} days")

## Step 8 — Cross-Case Comparison

Differentiated outputs — not a generic template response.

In [ ]:
def get_metrics(r):
    L = r['layers']
    tm = L.get('L2_TrialMatching', {})
    top_score = tm['top_matches'][0]['match_score'] if tm.get('top_matches') else 0
    return {
        'd': L['L1_Sentinel']['d_value'],
        'urgency': L['L2_MedGemma']['urgency'],
        'agid': L['L2_Trinity']['consensus_agid'].replace('AGID-', ''),
        'score': top_score,
        'cost': L.get('L2_5_Value', {}).get('total_cost_usd', 0),
        'days': L.get('L2_5_Value', {}).get('total_duration_days', 0),
    }

ma, mb, mc = get_metrics(result_a), get_metrics(result_b), get_metrics(result_c)

print(f"{'='*68}")
print(f"{'CROSS-CASE COMPARISON':^68}")
print(f"{'='*68}")
print(f"{'Metric':<22} {'Case A (ZH/NSCLC)':<22} {'Case B (EN/Stem)':<20} {'Case C (TH/PD)'}")
print(f"{'-'*68}")
for label, k, fmt in [('D-value','d','{:.3f}'),('Urgency','urgency','{}'),('Top AGID','agid','{}'),
                       ('Trial score','score','{:.2f}'),('Cost (USD)','cost','${:,.0f}'),('Duration (d)','days','{}')]:
    print(f"{label:<22} {fmt.format(ma[k]):<22} {fmt.format(mb[k]):<20} {fmt.format(mc[k])}")
print(f"{'='*68}")
scores = {ma['score'], mb['score'], mc['score']}
print(f"\nScore differentiation: {len(scores) >= 2} ({', '.join(str(s) for s in sorted(scores, reverse=True))})")

## Step 9 — Noise Input Safety Test

Non-medical input → L1 REVIEW → Trinity SOFT_CONFLICT → no crash.

In [ ]:
result_noise = run_full_pipeline('What is the weather today?', 'auto')
L = result_noise['layers']
print(f"  L1 status:      {L['L1_Sentinel']['status']}")
print(f"  Trinity status: {L.get('L2_Trinity', {}).get('status')}")
print(f"  AGID:           {L.get('L2_Trinity', {}).get('consensus_agid')}")
print(f"  Warning:        {result_noise.get('warning', 'None')}")
print('  Graceful degradation: no crash.')

## Step 10 — Launch Gradio UI (Optional)

Launches the full 4-tab interactive demo. Requires the full `app.py` from the GitHub repository.

In [ ]:
# Optional: launch full Gradio UI from repo
# Clone repo first: !git clone https://github.com/Smithl-Lin/codex-public.git
# Then:
# import sys; sys.path.insert(0, 'codex-public/0224 MedGamma/amani-medgemma-v1/amani-medgemma')
# from app import build_gradio_app
# ui = build_gradio_app()
# ui.launch(share=True, server_name='0.0.0.0', server_port=7860)

print('To launch Gradio UI: uncomment lines above after cloning the repo.')
print('Inline pipeline above runs independently without any external files.')

---

## Summary

| Component | Details |
|-----------|----------|
| Medical AI | Google MedGemma 1.5 4B (`google/medgemma-1.5-4b-it`) |
| Languages | ZH / AR / TH / EN |
| Cases | A: NSCLC 0.96 · B: StemCell 0.80 · C: PD/BCI 0.60 |
| Audit | 3-round multi-role: Builder + Observer + Verifier ✅ |
| License | Apache 2.0 |

**AMANI Sovereign Medical Routing Protocol — Kaggle AI in Medicine 2026**